# Today's to-do functions:
* Get ALL the strike's margins and price

In [1]:
# imports
from z_helper import *
util.startLoop()

ib = get_connected('snp', 'live')

# from json
a = assign_var('snp') + assign_var('common')
for v in a:
    exec(v)
    
jup_disp_adjust() # adjust jupyter's display

with open(logpath+'test.log', 'w'):
    pass # clear the run log

util.logToFile(logpath+'test.log')

In [2]:
%%time

# Get all the chains
df_chains = pd.read_pickle(fspath+'chains.pkl')

#... Get the underlying prices of the chains

undList = df_chains[['symbol', 'undId']].drop_duplicates().reset_index(drop=True)

und_cs = [Contract(conId=c) for c in list(undList.undId)]
und_blks = [und_cs[i: i+blk] for i in range(0, len(und_cs), blk)]

qc_blks = []  # qualified contract blocks
for u in und_blks:
    req = ib.qualifyContractsAsync(*u)
    qc_blks.append(ib.run(asyncio.wait_for(req, 12)))

t_blks = [] # tickers
for q in qc_blks:
    req = ib.reqTickersAsync(*q)
    t_blks.append(ib.run(asyncio.wait_for(req, 15)))

df_undPrice = pd.DataFrame.from_dict({t.contract.conId: t.marketPrice() 
                        for tblk in t_blks 
                        for t in tblk}, 'index', columns={'undPrice'})

df1 = df_chains.assign(undId=df_chains.undId.astype('int32')) \
        .set_index('undId') \
        .join(df_undPrice) \
        .rename_axis('undId') \
        .reset_index()

# get dte and remove those greater than maxdte
df1 = df1.assign(dte=df1.expiry.apply(get_dte))

# replace dte with 1 for dte <= 0
df_chains.loc[df1.dte <=0,  'dte'] = 1
df2 = df1[df1.dte <= maxdte]

# assign right
df3 = df2.assign(right=np.where(df2.strike >= df2.undPrice, 'C', 'P'))

# get the standard deviations
df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')

# generate std dataframe
dfo = df_ohlcs[['symbol', 'stDev']]  # lookup dataframe
dfo = dfo.assign(dte=dfo.groupby('symbol').cumcount()) # get the cumulative count for location as dte
dfo.set_index(['symbol', 'dte'])

dfd = df3[['symbol', 'dte']]  # data to be looked at
dfd = dfd.drop_duplicates()  # remove duplicates

df_std = dfd.set_index(['symbol', 'dte']).join(dfo.set_index(['symbol', 'dte']))

# join to get std in chains
df4 = df3.set_index(['symbol', 'dte']).join(df_std).reset_index()

# remove the calls and puts near strike price
c_mask = (df4.right == 'C') & (df4.strike > df4.undPrice + 2*df4.stDev)
p_mask = (df4.right == 'P') & (df4.strike < df4.undPrice + 2*df4.stDev)
df5 = df4[c_mask | p_mask].reset_index(drop=True)

# options and orders
opts = [Option(i.symbol, i.expiry, i.strike, i.right, 'SMART') for i in df5[['symbol', 'expiry', 'strike', 'right']].itertuples()]

if market == 'snp':
    ords = [Order(action='SELL', orderType='MKT', totalQuantity=1, whatIf=True) for r in df5.lot]
else:
    ords = [Order(action='SELL', orderType='MKT', totalQuantity=r, whatIf=True) for r in df5.lot]

Wall time: 30.7 s


In [3]:
%%time

# ... get the price and margins
opt_blks = [opts[i: i+blk] for i in range(0, len(opts), blk)]

qo_blks = [] # qualified option contract blocks
for o in opt_blks:
    req = ib.qualifyContractsAsync(*o)
    qo_blks.append(ib.run(asyncio.wait_for(req, 12)))

Wall time: 20min 41s


In [ ]:
%%time
to_blks = [] # tickers
for q in qo_blks:
    req = ib.reqTickersAsync(*q)
    to_blks.append(ib.run(asyncio.wait_for(req, 15)))

In [ ]:
%%time
und_blks = [und_cs[i: i+blk] for i in range(0, len(und_cs), blk)]

qc_blks = []  # qualified contract blocks
for u in und_blks:
    req = ib.qualifyContractsAsync(*u)
    qc_blks.append(ib.run(asyncio.wait_for(req, 12)))

t_blks = [] # tickers
for q in qc_blks:
    req = ib.reqTickersAsync(*q)
    t_blks.append(ib.run(asyncio.wait_for(req, 15)))

m_blks = [] # margins
for q in qc_blks:
    for m in q:
        req = ib.whatIfOrderAsync(m, Order(action='SELL', orderType='MKT', totalQuantity=1, whatIf=True))
        m_blks.append(ib.run(asyncio.wait_for(req, 2)))

In [ ]:
m_blks

In [ ]:
# get the chains
# get the underlying prices
# get the ohlcs and the stdev
# 

In [ ]:
qc_blks[0][0]

In [ ]:
# {con: symbol} dictionary
con_sym_dict = undList.set_index('undId').to_dict()['symbol']

In [ ]:
df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')

#.... Get option prices of sized_snp
df_sized = pd.read_pickle(fspath+'sized_snp.pkl')
opt_cs = [Contract(conId=c) for c in list(df_sized.optId)]

In [ ]:
cs = opt_cs[:100]
req = ib.qualifyContractsAsync(*cs)
qual_cs = ib.run(asyncio.wait_for(req, 12))

In [ ]:
qual_cs[:5]

In [ ]:
cs = qual_cs

In [ ]:
ticks = ib.reqTickers(*cs)

In [ ]:
ticks

In [ ]:
# async function to get ticker for blocks with timeout
async def get_ticks_Async(ib, cs):
    '''gets the ticker with a timeout in blocks of 50
    Arg:
        (ib) as connection object
        (cs) as contracts in block of 50
    Returns:
        ticks
    '''
    try:
        aw = await asyncio.wait_for(ib.reqTickersAsync(*cs, regulatorySnapshot=False), timeout=5)
    except asyncio.TimeoutError: # fails the timeout
        return np.nan
    
    # success!!
    return aw

In [ ]:


async def und_coro(und_cs):
    '''Async coroutine for underlying contract tickers'''
    t = await get_ticks_Async(ib, und_cs)
    return t
#     with tqdm(total=len(und_cs), file=sys.stdout, unit = ' tickers' ) as tqm:
#         for c in und_cs:
#             tqm.set_description(f"Ticker for {con_sym_dict[c.conId]}")
#             t.update()
#             tqm.update(1)

ticks=asyncio.run(und_coro(und_cs))

In [ ]:
und_cs[0:50]

In [ ]:
with tqdm(total=len(und_cs), file=sys.stdout, unit = ' tickers' ) as tqm:
    for c in und_cs:
        tqm.set_description(f"Ticker for {con_sym_dict[c.conId]}")
        tqm.update(1)
        ib.sleep(2)

In [ ]:
ib.accountSummary()

In [ ]:
df_sized = pd.read_pickle(fspath+'sized_snp.pkl')
assignment_limit = snp_assignment_limit

In [ ]:
p = util.df(ib.portfolio()) # portfolio table

# extract option contract info from portfolio table
if p is not None:  # there are some contracts in the portfolio
    dfp = pd.concat([p, util.df([c for c in p.contract])[util.df([c for c in p.contract]).columns[:7]]], axis=1).iloc[:, 1:]
    dfp = dfp.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

    # extract the options
    dfpo = dfp[dfp.secType == 'OPT']

    # get unique symbol, lot and underlying
    df_lu = df_sized[['symbol', 'lot', 'undId', 'undPrice']].groupby('symbol').first()

    # integrate the options with lot and underlying
    dfp1 = dfpo.set_index('symbol').join(df_lu).reset_index()

    # correct the positions for nse
    if exchange == 'NSE':
        dfp1 = dfp1.assign(position=dfp1.position/dfp1.lot)

    # get the total position for options
    dfp1 = dfp1[['symbol', 'position']].groupby('symbol').sum()

    # ... integrate postion and lots and underlyings
    # _______________________________________________
    dfrq1 = df_lu.join(dfp1)
else:
    print('There is nothing in the portfolio')
    dfrq1 = df_lu
    dfrq1['position'] = 0

In [ ]:
# fill in the other columns
dfrq1 = dfrq1.assign(assVal=dfrq1.position*dfrq1.lot*dfrq1.undPrice)
dfrq1 = dfrq1.assign(assQty=-(assignment_limit/dfrq1.lot/dfrq1.undPrice).astype('int32'))
dfrq1 = dfrq1.assign(mgnQty=-(assignment_limit/dfrq1.lot/dfrq1.undPrice/.225).astype('int32'))
dfrq1 = dfrq1.assign(remq=(dfrq1.position-dfrq1.mgnQty))
dfrq1 = dfrq1.assign(remq=dfrq1.remq.fillna(0))

In [ ]:
dfrq1

In [ ]:
def print_something():
    print('This is just something')

In [ ]:
ib.orderStatusEvent -= print_something

In [ ]:
ib.disconnectedEvent += print_something

In [ ]:
ib.disconnect()

In [ ]:
ib.disconnectedEvent -= print_something

In [ ]:
# input files for the functions
df_chains = pd.read_pickle(fspath+'chains.pkl')
df_sized = pd.read_pickle(fspath+'sized_nse.pkl')
assignment_limit = nse_assignment_limit

In [ ]:
# Preparatory inputs for df_sized
df_chains = pd.read_pickle(fspath+'chains.pkl')
df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')



filt1 = "sdmult" # fr=fallrise*, sdmult=standard deviation, , hilo52 = hilo52
filt2 = "all" # all=all options*, putsonly = puts only

In [ ]:
%%time
#... Remove chains not meeting put and call std filter

# get dte and remove those greater than maxdte
df_chains = df_chains.assign(dte=df_chains.expiry.apply(get_dte))                    
df_chains = df_chains[df_chains.dte <= maxdte]

# replace dte with 1 for dte <= 0
df_chains.loc[df_chains.dte <=0,  'dte'] = 1

# generate std dataframe
df = df_ohlcs[['symbol', 'stDev']]  # lookup dataframe
df = df.assign(dte=df.groupby('symbol').cumcount()) # get the cumulative count for location as dte
df.set_index(['symbol', 'dte'])

df1 = df_chains[['symbol', 'dte']]  # data to be looked at
df2 = df1.drop_duplicates()  # remove duplicates

df_std = df2.set_index(['symbol', 'dte']).join(df.set_index(['symbol', 'dte']))

# join to get std in chains
df_chainstd = df_chains.set_index(['symbol', 'dte']).join(df_std).reset_index()

# first stocks
und_contracts = [Stock(symbol, exchange=exchange) for symbol in df_chainstd.symbol.unique()]
und_quals = ib.qualifyContracts(*und_contracts)

# then indexes
idxs = set(df_chainstd.symbol.unique())-{u.symbol for u in und_quals}
und_idxs = [Index(symbol, exchange=exchange) for symbol in idxs]
und_idx_quals = ib.qualifyContracts(*und_idxs)

und_quals = und_idx_quals+und_quals

tickers = ib.reqTickers(*und_quals)

uprice_dict = {u.contract.conId: u.marketPrice() for u in tickers}

df_chainstd = df_chainstd.assign(undPrice=df_chainstd.undId.astype('int').map(uprice_dict))

# make puts and calls dataframe with std filter
df_puts = df_chainstd[df_chainstd.strike < (df_chainstd.undPrice-(df_chainstd.stDev*putstdmult))]
df_puts = df_puts.assign(right = 'P')

df_calls = df_chainstd[df_chainstd.strike > (df_chainstd.undPrice+(df_chainstd.stDev*callstdmult))]
df_calls = df_calls.assign(right = 'C')

df_opt = pd.concat([df_puts, df_calls], sort=False).reset_index(drop=True)

# get lo52 and hi52
df_opt = df_opt.set_index('symbol').join(df_ohlcs.groupby('symbol')
                                         .close.agg(['min', 'max'])
                                         .rename(columns={'min': 'lo52', 'max': 'hi52'})).reset_index()

# make (df and dte) tuple for fallrise
tup4fr = [(df_ohlcs[df_ohlcs.symbol == s.symbol], s.dte) 
          for s in df_opt[['symbol', 'dte']].drop_duplicates().itertuples()]

# get the fallrise and put it into a dataframe
fr = [fallrise(*t) for t in tup4fr]
df_fr = pd.DataFrame(fr, columns=['symbol', 'dte', 'fall', 'rise' ])

# merge with df_opt
df_opt = pd.merge(df_opt, df_fr, on=['symbol', 'dte'])

# make reference strikes from filter
if filt1 == 'sdmult':
    df_opt = df_opt.assign(strikeRef = np.where(df_opt.right == 'P', 
                                            df_opt.undPrice-putstdmult*df_opt.stDev, 
                                            df_opt.undPrice+callstdmult*df_opt.stDev))
elif filt1 == 'hilo52':
    df_opt = df_opt.assign(strikeRef = np.where(df_opt.right == 'P', 
                                            df_opt.lo52 - df_opt.stDev, 
                                            df_opt.hi52 + df_opt.stDev))    
else: # default (fallrise)
    df_opt = df_opt.assign(strikeRef = np.where(df_opt.right == 'P', 
                                                df_opt.undPrice-df_opt.fall, 
                                                df_opt.undPrice+df_opt.rise))    

In [ ]:
df_opt[(df_opt.symbol == 'APOLLOTYR') & (df_opt.dte == 20) & (df_opt.right == 'C')].head()

In [ ]:
df_ohlcs[df_ohlcs.symbol=='NIFTY50'].close.rolling(20).mean().iloc[20]

In [ ]:
df_acc=df_ohlcs[df_ohlcs.symbol=='ACC'].loc[:252/2]

In [ ]:
df_acc.close.rolling(20).mean()

In [ ]:
66.4+27.10

In [ ]:
df_chains = pd.read_pickle(fspath+'chains_nse.pkl')

#...undprice refresh
syms = list(df_chains.symbol.unique())

# get the contracts and undprice
qual_stks = ib.qualifyContracts(*[Stock(s, exchange=exchange) for s in syms])
idxs = {s for s in syms if s not in {q.symbol for q in qual_stks}}
qual_idxs = ib.qualifyContracts(*[Index(s, exchange=exchange) for s in idxs])
qual_unds = qual_stks+qual_idxs
undtickers = ib.reqTickers(*qual_unds)
uprice_dict = {u.contract.conId: u.marketPrice() for u in undtickers}